### Dans ce carnet Jupyter, j'ai entrepris de tester un modèle LLM [vigogne-2-7b-chat](https://huggingface.co/bofenghuang/vigogne-2-7b-chat) spécialement conçu pour le français. Initialement, j'ai évalué le modèle en lui posant quelques questions. Par la suite, j'ai employé LangChain pour adapter les réponses du modèle en fonction du contexte. J'ai également téléchargé un document PDF et posé des questions à propos de son contenu.

In [1]:
pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.3 MB/s eta 0:00:00


In [2]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 5.3 MB/s eta 0:00:00


In [3]:
!pip install -qqq git+https://github.com/bofenghuang/vigogne


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.

In [ ]:
# Uncomment if necessary
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
from typing import Dict, List, Optional
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer

In [5]:
model_name_or_path = "bofenghuang/vigogne-2-7b-chat"
revision = "v2.0"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, revision=revision, padding_side="right", use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    revision=revision,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    #low_cpu_mem_usage=True,
)

streamer = TextStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00007.bin:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

pytorch_model-00007-of-00007.bin:   0%|          | 0.00/1.66G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
def chat(
    query: str,
    history: Optional[List[Dict]] = None,
    temperature: float = 0.7,
    top_p: float = 1.0,
    top_k: float = 0,
    repetition_penalty: float = 1.1,
    max_new_tokens: int = 1024,
    **kwargs,
):
    if history is None:
        history = []

    history.append({"role": "user", "content": query})

    input_ids = tokenizer.apply_chat_template(history, add_generation_prompt=True, return_tensors="pt").to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            top_p=top_p,
            top_k=top_k,
            repetition_penalty=repetition_penalty,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.eos_token_id,
            **kwargs,
        ),
        streamer=streamer,
        return_dict_in_generate=True,
    )

    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    history.append({"role": "assistant", "content": generated_text})

    return generated_text, history

In [ ]:
# 1st round
response, history = chat("Un escargot parcourt 100 mètres en 5 heures. Quelle est sa vitesse ?", history=None)

La formule pour calculer la vitesse est :

Vitesse = Distance / Temps

On connait la distance (100 m) et le temps (5 heures). Pour trouver la vitesse, il faut diviser la distance par le temps :

Vitesse = 100 m / 5 h

Maintenant, convertir les unités de temps en heures:

Vitesse = 100 m/h * 60 min/h = 6 km/h

La vitesse de l'escargot est donc de 6 km/h.


In [ ]:
# 2nd round
response, history = chat("Le SCR relatif au risque de souscription vie est calculé comment?", history=history)

L'indice SCR (Scrutin de Contrôle Rationnel) représente une mesure du risque de souscription à un titre. Il s'agit d'un ratio qui compare le coût de capital qu'une entreprise doit souscrire pour financer ses activités avec la valeur de son actif net. Plus ce ratio est faible, moins le risque de souscription est élevé pour les investisseurs.

Le calcul de l'SCR est effectué en divisant la valeur nette des actifs tangibles de l'entreprise par ses obligations subordonnées ou subordinées. Voici la formule générale pour calculer l'SCR :

SCR = (Valorisation nette des actifs + Actif financier exprimé en francs) / Nombre de parts subordonnées

Pour les titres de créances subordonnées, le SCR serait calculé comme suit :

SCR = (Valorisation nette des actifs - Dettes subordonnées) / Nombre de parts subordonnées

Cependant, il existe également une version simplifiée du SCR, appelée SCR-D, qui ne prend pas en compte la dette subordonnée dans le calcul de l'indice. Cette méthode simplifie la formu

In [7]:
pip install langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=4031da05910890ab89328e587e3eb58c93217c981972d9f46c0544a0e48f0d90
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [8]:
import logging
from functools import partial
from typing import Optional
import fire
import torch
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, WebBaseLoader,PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import VLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [9]:
stuff_prompt_template = """<|system|>: Vous êtes un assistant IA qui répond à la question posée à la fin en utilisant le contexte suivant. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.
<|user|>: {context}
Question: {question}
<|assistant|>:"""
refine_initial_prompt_template = """<|system|>: Vous êtes un assistant IA qui répond à la question posée à la fin en utilisant le contexte suivant. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.
<|user|>: {context_str}
Question: {question}
<|assistant|>:"""
refine_prompt_template = """<|system|>: Vous êtes un assistant IA qui répond à la question posée à la fin en utilisant le contexte suivant. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, n'essayez pas d'inventer une réponse.
<|user|>: Question: {question}
Réponse existante: {existing_answer}
Vous avez la possibilité d'affiner la réponse existante (seulement si nécessaire) avec plus de contexte ci-dessous.
------------
{context_str}
------------
Veuillez affiner la réponse existante avec le contexte fournie. Si le contexte n'est pas utile, renvoyez la réponse existante.
<|assistant|>:"""

In [ ]:
input_file= None,
web_url: Optional[str] = None,
chunk_size= 500,
chunk_overlap=0,
chain_type: str = "stuff",
embedding_model_name_or_path: str = "dangvantuan/sentence-camembert-base",
llm_model_name_or_path: str = "bofenghuang/vigogne-2-7b-chat",
llm_revision= "main"
initial_question= None

In [10]:
llm_revision= "main"
initial_question= None

In [11]:
chunk_size= 500
chunk_overlap=0
embedding_model_name_or_path= "dangvantuan/sentence-camembert-base"
llm_model_name_or_path = "bofenghuang/vigogne-2-7b-chat"

In [12]:
!wget https://plubioclimatique.paris.fr/projet/datas/pdf/RP_EIE.pdf

--2024-01-24 00:48:35--  https://plubioclimatique.paris.fr/projet/datas/pdf/RP_EIE.pdf
Resolving plubioclimatique.paris.fr (plubioclimatique.paris.fr)... 194.153.110.192
Connecting to plubioclimatique.paris.fr (plubioclimatique.paris.fr)|194.153.110.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14580566 (14M) [application/pdf]
Saving to: ‘RP_EIE.pdf’

RP_EIE.pdf          100%[===================>]  13.90M   785KB/s    in 19s     

2024-01-24 00:48:54 (762 KB/s) - ‘RP_EIE.pdf’ saved [14580566/14580566]



In [13]:
from langchain.retrievers.multi_vector import MultiVectorRetriever



Ces lignes de code en Python créent une instance loader de la classe PyPDFLoader pour charger le contenu d'un fichier PDF situé à l'emplacement '/content/RP_EIE.pdf', puis utilisent la méthode load() de cette instance pour lire et stocker les données du fichier PDF dans la variable data.

In [ ]:

loader = PyPDFLoader('/content/RP_EIE.pdf')
data = loader.load()


Création et utilisation d'un séparateur de texte :



*   Un objet text_splitter est créé à partir de la classe RecursiveCharacterTextSplitter. Cet objet est configuré pour diviser le texte en segments de taille chunk_size avec un chevauchement de chunk_overlap.
*   La méthode split_documents() de text_splitter est utilisée pour diviser les données textuelles stockées dans data en plusieurs segments plus petits, et les segments résultants sont stockés dans la variable all_splits.

Configuration et création d'un modèle d'embedding :


*   Un dictionnaire encode_kwargs est défini avec la clé normalize_embeddings réglée sur False, indiquant que les embeddings ne seront pas normalisés lors de l'encodage.

*   Un objet embedding_model est créé à partir de la classe HuggingFaceEmbeddings. Ce modèle est configuré pour utiliser un modèle spécifique d'embedding de langage (indiqué par embedding_model_name_or_path). Les paramètres d'encodage (non normalisés dans ce cas) sont passés via encode_kwargs.



In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
all_splits = text_splitter.split_documents(data)

encode_kwargs = {"normalize_embeddings": False}
embedding_model = HuggingFaceEmbeddings(
    model_name=embedding_model_name_or_path,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

.DS_Store:   0%|          | 0.00/6.15k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/463 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
#stocker les données vectorisés dans la base de données vectorielle FAISS
vectorstore = FAISS.from_documents(all_splits, embedding_model)

Ce code configure donc un pipeline de génération de texte avec des paramètres spécifiques pour le modèle et la génération, optimisé pour l'utilisation sur un GPU avec une utilisation de mémoire réduite.

In [ ]:
llm = HuggingFacePipeline.from_model_id(
        model_id=llm_model_name_or_path,
        task="text-generation",
        device=0,
        model_kwargs={
            "torch_dtype": torch.float16,
            "low_cpu_mem_usage": True,
        },
        pipeline_kwargs={
            "revision": llm_revision,
            # "trust_remote_code": True,
            "temperature": 0.1,
            "max_new_tokens": 1024,
            "repetition_penalty": 1.1,
        },
    )

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Ce code configure une chaîne de traitement pour la question-réponse (QA) avec récupération d'informations (RetrievalQA) en utilisant un modèle de langage à grande échelle (LLM) et un système de récupération basé sur un magasin de vecteurs. Voici les détails clés :

Création d'un objet PromptTemplate :

*   prompt: Un objet PromptTemplate est créé avec un template spécifié dans stuff_prompt_template. Ce template détermine la structure de la requête envoyée au modèle de langage.
*   input_variables: Spécifie les variables d'entrée pour le template, ici "context" et "question".


Configuration des arguments pour le type de chaîne :

*   chain_type_kwargs: Un dictionnaire contenant le prompt configuré, à utiliser dans la chaîne de traitement.

Création de la chaîne de QA avec récupération (RetrievalQA) :


*   llm: Le modèle de langage à grande échelle préalablement configuré.

*   retriever: Un objet de récupération créé à partir d'un magasin de vecteurs (vectorstore.as_retriever()), utilisé pour rechercher des documents pertinents en fonction des requêtes.


*   chain_type: Le type de chaîne, ici "stuff", qui définit la logique de traitement spécifique.
*   chain_type_kwargs: Les arguments configurés pour ce type de chaîne.


*   return_source_documents: Un booléen indiquant si les documents sources doivent être retournés avec les réponses.

In [ ]:
prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": prompt}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    return_source_documents=True,
)

### Tester

In [ ]:
question='quelles sont les caneaux à paris'
result = qa_chain({"query": question})

In [ ]:
result

{'query': 'quelles sont les caneaux à paris',
 'result': " À Paris, il existe trois canaux :\n\n1. Le canal de l'Ourcq : de Pavillons-sous-Bois jusqu'au bassin de la Villette (11 km)\n2. Le canal Saint-Denis : de 6,6 km avec 7 écluses à double sas, pouvant supporter des bateaux de 600 tonnes à l'amont et 1 000 tonnes à l'aval\n3. Le canal Saint-Martin : de 4,5 km, dont 2 km en souterrain, avec 9 écluses dont 4 doubles, limitant les bateaux à 350 tonnes",
 'source_documents': [Document(page_content='d’eau  dans la ville. Sa réouverture en amont de Paris  est déjà effective sur certains tronçons. Le projet tend à se \npoursuivre en plusieurs phases  : études préliminaires (2021 -2023), conception (2024) et premiers travaux dans le parc \nKellermann ( fin 2024, 2025).  \n \n Les canaux  \nOn compte dans Paris trois ouvrag es, dont : \n\uf0b7 Deux  ouvrages  à grand gabarit  : \no Le canal de l’Ourcq de Pavillons -sous -Bois jusqu’au bassin de la Villette (11 km) ;', metadata={'source': '/